# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1: Marta Castillo Galán
*   Alumno 2: Unai Marín Etxebarria
*   Alumno 3: Laura Molinos Mayo
*   Alumno 4: Raúl Murillo Gallego






---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

In [ ]:
#if IN_COLAB:
  #%pip install gym==0.17.3
  #%pip install git+https://github.com/Kojoley/atari-py.git
  #%pip install keras-rl2==1.0.5
  #%pip install tensorflow==2.8
#else:
  #%pip install gym==0.17.3
  #%pip install git+https://github.com/Kojoley/atari-py.git
  #%pip install pyglet==1.5.0
  #%pip install h5py==3.1.0
  #%pip install Pillow==9.5.0
  #%pip install keras-rl2==1.0.5
  #%pip install Keras==2.2.4
  #%pip install tensorflow==2.5.3
  ##%pip install torch==2.0.1
  #%pip install agents==1.4.0

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [1]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

from tensorflow.keras.optimizers import RMSprop

#### Configuración base

In [2]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [3]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

1. Implementación de la red neuronal

In [4]:
def build_model(window_length, input_shape, nb_actions):
    model = Sequential()
    model.add(Permute((2, 3, 1), input_shape=(window_length,) + input_shape))  # (window, 84, 84) → (84, 84, window)
    model.add(Convolution2D(32, (8, 8), strides=(4, 4), activation='relu'))
    model.add(Convolution2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(nb_actions, activation='linear'))
    return model

2. Implementación de la solución DQN

In [5]:
# Preparamos memoria y política
memory = SequentialMemory(limit=100000, window_length=WINDOW_LENGTH)
#policy = EpsGreedyQPolicy()  # Exploración simple
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(),
                              attr='eps',
                              value_max=1.0,
                              value_min=0.1,
                              value_test=0.05,
                              nb_steps=50000)  # más suave

# Creamos el modelo
model = build_model(WINDOW_LENGTH, INPUT_SHAPE, nb_actions)
print(model.summary())

# Creamos el procesador
processor = AtariProcessor()

#Creamos el agente
dqn = DQNAgent(model=model,
               nb_actions=nb_actions,
               policy=policy,
               memory=memory,
               processor=processor,
               nb_steps_warmup = 10000,
               gamma=0.99,
               target_model_update=5000,
               train_interval=1,
               delta_clip=1.0)

dqn.compile(Adam(learning_rate=0.0001), metrics=['mae'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1606144   
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3

In [6]:
# Entrenamiento inicial
# (0.0.0.3) Pasamos de 20.000
#checkpoint = ModelIntervalCheckpoint('dqn_spaceinvaders_weights_step{step}.h5f',interval=5000)
#dqn.fit(env, nb_steps=20000, visualize=False, verbose=2, callbacks=[checkpoint])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=2)

Training for 50000 steps ...


C:\Users\Rociniel\Anaconda3\envs\miar_rl\lib\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


   420/50000: episode: 1, duration: 2.231s, episode steps: 420, steps per second: 188, episode reward:  6.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.495 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   960/50000: episode: 2, duration: 2.731s, episode steps: 540, steps per second: 198, episode reward:  6.000, mean reward:  0.011 [ 0.000,  1.000], mean action: 2.406 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1384/50000: episode: 3, duration: 2.129s, episode steps: 424, steps per second: 199, episode reward:  1.000, mean reward:  0.002 [ 0.000,  1.000], mean action: 2.528 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1858/50000: episode: 4, duration: 2.910s, episode steps: 474, steps per second: 163, episode reward:  2.000, mean reward:  0.004 [ 0.000,  1.000], mean action: 2.432 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  3130/50000: episode: 5, duration: 9.150s, episode steps: 1272, steps per second: 1

C:\Users\Rociniel\Anaconda3\envs\miar_rl\lib\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


 10769/50000: episode: 16, duration: 133.633s, episode steps: 827, steps per second:   6, episode reward: 13.000, mean reward:  0.016 [ 0.000,  1.000], mean action: 2.530 [0.000, 5.000],  loss: 0.006998, mae: 0.040853, mean_q: 0.065717, mean_eps: 0.813079
 11245/50000: episode: 17, duration: 82.619s, episode steps: 476, steps per second:   6, episode reward:  8.000, mean reward:  0.017 [ 0.000,  1.000], mean action: 2.437 [0.000, 5.000],  loss: 0.007065, mae: 0.041876, mean_q: 0.072255, mean_eps: 0.801883
 11668/50000: episode: 18, duration: 66.126s, episode steps: 423, steps per second:   6, episode reward:  5.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.182 [0.000, 5.000],  loss: 0.007236, mae: 0.042812, mean_q: 0.076292, mean_eps: 0.793792
 12775/50000: episode: 19, duration: 179.129s, episode steps: 1107, steps per second:   6, episode reward: 17.000, mean reward:  0.015 [ 0.000,  1.000], mean action: 2.547 [0.000, 5.000],  loss: 0.006635, mae: 0.043497, mean_q: 0.0860

In [8]:
#Evaluación en modo test
scores = dqn.test(env, nb_episodes=10, visualize=False)
print("Media de recompensa:", np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 9.000, steps: 811
Episode 2: reward: 21.000, steps: 1290
Episode 3: reward: 9.000, steps: 771
Episode 4: reward: 13.000, steps: 950
Episode 5: reward: 17.000, steps: 863
Episode 6: reward: 12.000, steps: 1139
Episode 7: reward: 4.000, steps: 540
Episode 8: reward: 6.000, steps: 666
Episode 9: reward: 9.000, steps: 831
Episode 10: reward: 8.000, steps: 731
Media de recompensa: 10.8


In [9]:
#Guardado de pesos
dqn.save_weights('dqn_spaceinvaders_weights.h5f', overwrite=True)

In [ ]:
## Testing part to calculate the mean reward
#weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
#dqn.load_weights(weights_filename)
#dqn.test(env, nb_episodes=10, visualize=False)

3. Justificación de los parámetros seleccionados y de los resultados obtenidos

---